In [4]:
import pandas as pd
import numpy as np
import random

**Exercise 1**

In [5]:
def load_Xy(path):

    #Use built-in pandas function to read csv file into a dataframe
    df = pd.read_csv(path)

    #Convert the dataframe to a numpy array
    data_array = df.to_numpy()

    #Separate the features X from the labels y
    X,y = data_array[:,:-1], data_array[:,-1]

    #Replace the 0 label with -1
    y[y==0] = -1 

    #return X and y
    return X,y


In [6]:
#Call the function and load the data
X,y = load_Xy('data.csv')

In [7]:
#Count how many malicious apps there are
# Filter the y array only where y == -1, then get its dimensions, and then its length
y[ y==-1 ].shape[0]

14632

In [8]:
#Let's count how many non-zero entries are in X
non_zero_entries = X[X!=0].shape[0]

#Total elements in X
tot_entries = X.shape[0]*X.shape[1]

print("Non-zero entries: {}".format(non_zero_entries))
print("Total entries: {}".format(tot_entries))
print("Ratio: {}".format(non_zero_entries/tot_entries))

Non-zero entries: 277180
Total entries: 2522552
Ratio: 0.10988078739308446


Only the $10\%$ of the elements of the matrix $X$ are non-zero, thus $X$ has a sparsity of $90\%$.

In [9]:
np.unique(X)

array([0, 1], dtype=int64)

**Exercise 2**

In [46]:
def train_test_random(X,y):    
    r = 0.8
    train_size = int(y.size*r)
    index = range(y.size)
    index_train = random.sample(index, train_size)
    index_train.sort()
    y_train = []
    y_test = []
    X_train = []
    X_test = []
    for i in range(y.size):
        if i in index_train:
            y_train.append(y[i])
            X_train.append(X[i,:])
        else:
            y_test.append(y[i])
            X_test.append(X[i,:])
    X_train = np.vstack(X_train)
    X_test = np.vstack(X_test)
    return X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_random(X,y)

**Exercise 3**

**Exercise 4**

**Exercise 5**

**Exercise 6**